In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sample_submission.csv
/kaggle/input/test_identity.csv
/kaggle/input/train_identity.csv
/kaggle/input/test_transaction.csv
/kaggle/input/train_transaction.csv


In [3]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [4]:
Identity = pd.read_csv('/kaggle/input/train_identity.csv')
Transaction = pd.read_csv('/kaggle/input/train_transaction.csv')

In [5]:
Identity.shape, Transaction.shape

((144233, 41), (590540, 394))

In [6]:
df = Transaction.merge(Identity, how='left', left_on='TransactionID', right_on='TransactionID')

In [7]:
Transaction.columns

Index(['TransactionID', 'isFraud', 'TransactionDT', 'TransactionAmt',
       'ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5',
       ...
       'V330', 'V331', 'V332', 'V333', 'V334', 'V335', 'V336', 'V337', 'V338',
       'V339'],
      dtype='object', length=394)

In [8]:
Identity.columns

Index(['TransactionID', 'id_01', 'id_02', 'id_03', 'id_04', 'id_05', 'id_06',
       'id_07', 'id_08', 'id_09', 'id_10', 'id_11', 'id_12', 'id_13', 'id_14',
       'id_15', 'id_16', 'id_17', 'id_18', 'id_19', 'id_20', 'id_21', 'id_22',
       'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_28', 'id_29', 'id_30',
       'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38',
       'DeviceType', 'DeviceInfo'],
      dtype='object')

In [11]:
df.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M


In [12]:
from sklearn.model_selection import GroupShuffleSplit

X = df.drop(columns=['isFraud'])
y = df['isFraud']

gss = GroupShuffleSplit(test_size=0.2, n_splits=1, random_state=42)

split = gss.split(X, y, groups=X['card1'].astype(str) + "_" + X['addr1'].astype(str))

train_idx, test_idx = next(split)

X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

# MLflow

In [13]:
!pip install mlflow dagshub
import mlflow
import dagshub

dagshub.init(repo_owner='Givi-Modebadze', repo_name='Fraud_Detection_ML', mlflow=True)

experiment = "Fraud Detection XGBoost"
mlflow.set_experiment(experiment)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.0/29.0 MB 31.2 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 88.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.1/260.1 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 692.3/692.3 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: dacite
    Found existing installation: dacite 1.9.2
    Uninstalling dacite-1.9.2:
      

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=201afb53-984c-4763-830a-254d31f597b9&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=83f15ec060e9ad72756bcb6bb1a76cc66323b475d260439d9c7daf6400235327




Output()

Accessing as Givi-Modebadze

Initialized MLflow to track repo "Givi-Modebadze/Fraud_Detection_ML"

Repository Givi-Modebadze/Fraud_Detection_ML initialized!

2025/04/27 13:18:58 INFO mlflow.tracking.fluent: Experiment with name 'Fraud Detection XGBoost' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/03dda8169694417092defb08e8e6bf64', creation_time=1745759938765, experiment_id='3', last_update_time=1745759938765, lifecycle_stage='active', name='Fraud Detection XGBoost', tags={}>

# Cleaning

In [14]:
from sklearn.base import BaseEstimator, TransformerMixin

class DataCleaner(BaseEstimator, TransformerMixin):
    def __init__(self, drop_threshold: float):
        self.drop_threshold = drop_threshold
        self.cols_to_drop_ = []
        self.fill_values = {}

    def fit(self, X, y = None):
        self.drop_cols = [col for col in X.columns if X[col].isna().mean() >= self.drop_threshold]
        self.drop_cols.append('TransactionID')

        X_filtered = X.drop(columns=self.drop_cols)

        for col in X_filtered.columns:
            mode = X_filtered[col].mode()
            self.fill_values[col] = mode[0] if not mode.empty else np.nan
        return self

    def transform(self, X):
        X = X.copy()
        X = X.drop(columns=self.drop_cols)
        for col, value in self.fill_values.items():
            X[col] = X[col].fillna(value)
        return X

In [25]:
data_cleaner = DataCleaner(drop_threshold=0.9)

data_cleaner.fit(X_train, y_train)

X_cleaned = data_cleaner.transform(X_train)

with mlflow.start_run(run_name="XGBoost_Clean") as run:
    mlflow.sklearn.log_model(
        sk_model=data_cleaner,
        artifact_path="data_cleaner_model",
        registered_model_name="DataCleaner",
    )
    mlflow.log_param("drop_threshold", 0.8)
    mlflow.log_param("n_columns_after_cleaning", X_cleaned.shape[1])

2025/04/27 14:46:35 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2025/04/27 14:46:39 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'DataCleaner' already exists. Creating a new version of this model...
2025/04/27 14:46:42 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: DataCleaner, version 12
Created version '12' of model 'DataCleaner'.


🏃 View run XGBoost_Clean at: https://dagshub.com/Givi-Modebadze/Fraud_Detection_ML.mlflow/#/experiments/3/runs/6e037bf711a547e49adb76431a26a2ad
🧪 View experiment at: https://dagshub.com/Givi-Modebadze/Fraud_Detection_ML.mlflow/#/experiments/3


# Feature Engineer

In [35]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OrdinalEncoder

class FeatureEngineer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.encoder = None

    def fit(self, X, y=None):
        cat_cols = X.select_dtypes(include=['object', 'category']).columns
        self.cat_cols = cat_cols
        self.encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
        self.encoder.fit(X[cat_cols])
        return self

    def transform(self, X):
        X = X.copy()
        X[self.cat_cols] = self.encoder.transform(X[self.cat_cols])
        return X

In [36]:
from sklearn.pipeline import Pipeline

feature_engineer = Pipeline([
    ('feature_engineering', FeatureEngineer())
])

feature_engineer.fit(X_cleaned, y_train)

X_engineered = feature_engineer.transform(X_cleaned)

with mlflow.start_run(run_name="XGBoost_Feature_Engineering") as run:
    mlflow.sklearn.log_model(
        sk_model=feature_engineer,
        artifact_path="feature_engineer_model",
        registered_model_name="FeatureEngineer",
    )

    n_label_encoded_cols = len(X_cleaned.select_dtypes(include=['object', 'category']).columns)
    n_numeric_cols = len(X_cleaned.select_dtypes(include=['number']).columns)
    total_cols = X_cleaned.shape[1]
    
    n_onehot_cols = len([
        col for col in X_cleaned.select_dtypes(include=['object', 'category']).columns 
        if X_cleaned[col].nunique() <= 3
    ])
    
    mlflow.log_param("n_label_encoded_columns", n_label_encoded_cols)
    mlflow.log_param("n_numeric_columns_scaled", n_numeric_cols)
    mlflow.log_param("total_columns", total_cols)

2025/04/27 15:00:07 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2025/04/27 15:00:11 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'FeatureEngineer' already exists. Creating a new version of this model...
2025/04/27 15:00:14 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: FeatureEngineer, version 9
Created version '9' of model 'FeatureEngineer'.


🏃 View run XGBoost_Feature_Engineering at: https://dagshub.com/Givi-Modebadze/Fraud_Detection_ML.mlflow/#/experiments/3/runs/ea7d88809c464b26aae00605830abd3b
🧪 View experiment at: https://dagshub.com/Givi-Modebadze/Fraud_Detection_ML.mlflow/#/experiments/3


# Feature Selection

# Training

In [38]:
X_engineered.shape

(463215, 417)

In [39]:
from sklearn.pipeline import Pipeline
import xgboost as xgb
import mlflow.sklearn

data_cleaner = mlflow.sklearn.load_model('runs:/6e037bf711a547e49adb76431a26a2ad/data_cleaner_model')
feature_engineer = mlflow.sklearn.load_model('runs:/ea7d88809c464b26aae00605830abd3b/feature_engineer_model')

pipeline = Pipeline([
    ('cleaner', data_cleaner),
    ('feature_engineering', feature_engineer),
    ('xgboost', xgb.XGBClassifier(
        n_estimators=500,
        max_depth=6,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=-1,
        tree_method="hist",
        use_label_encoder=False,
        eval_metric='auc'
    ))
])

pipeline.fit(X_train, y_train)
y_prob_train = pipeline.predict_proba(X_train)[:, 1]
y_prob_test = pipeline.predict_proba(X_test)[:, 1]

from sklearn.metrics import roc_auc_score

with mlflow.start_run(run_name="XGBoost_Model") as run:
    mlflow.sklearn.log_model(
        sk_model=pipeline,
        artifact_path="xgboost_pipeline_model",
        registered_model_name="XGBoostPipelineModel"
    )
    
    mlflow.log_param("n_estimators", 500)
    mlflow.log_param("max_depth", 6)
    mlflow.log_param("learning_rate", 0.1)
    mlflow.log_param("subsample", 0.8)
    mlflow.log_param("colsample_bytree", 0.8)
    mlflow.log_param("random_state", 42)
    mlflow.log_param("tree_method", "hist")
    mlflow.log_param("n_jobs", -1)

    roc_auc_train = roc_auc_score(y_train, y_prob_train)
    roc_auc_test = roc_auc_score(y_test, y_prob_test)

    mlflow.log_metric("roc_auc_train", roc_auc_train)
    mlflow.log_metric("roc_auc_test", roc_auc_test)
    
    print("ROC AUC Score Train:", roc_auc_train)
    print("ROC AUC Score Test:", roc_auc_test)

2025/04/27 15:04:48 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'XGBoostPipelineModel'.
2025/04/27 15:04:51 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGBoostPipelineModel, version 1
Created version '1' of model 'XGBoostPipelineModel'.


ROC AUC Score Train: 0.9784898301406799
ROC AUC Score Test: 0.8906932410026578
🏃 View run XGBoost_Model at: https://dagshub.com/Givi-Modebadze/Fraud_Detection_ML.mlflow/#/experiments/3/runs/be546fd377354c0c8cffc461e1794961
🧪 View experiment at: https://dagshub.com/Givi-Modebadze/Fraud_Detection_ML.mlflow/#/experiments/3


# Training 2

In [41]:
from sklearn.pipeline import Pipeline
import xgboost as xgb
import mlflow.sklearn

data_cleaner = mlflow.sklearn.load_model('runs:/6e037bf711a547e49adb76431a26a2ad/data_cleaner_model')
feature_engineer = mlflow.sklearn.load_model('runs:/ea7d88809c464b26aae00605830abd3b/feature_engineer_model')

pipeline = Pipeline([
    ('cleaner', data_cleaner),
    ('feature_engineering', feature_engineer),
    ('xgboost', xgb.XGBClassifier(
        n_estimators=500,
        max_depth=5,
        learning_rate=0.05,
        subsample=0.85,
        colsample_bytree=0.7,
        reg_alpha=0.1,
        reg_lambda=1.0,
        random_state=42,
        n_jobs=-1,
        tree_method="hist",
        use_label_encoder=False,
        eval_metric='auc'
    ))
])

pipeline.fit(X_train, y_train)
y_prob_train = pipeline.predict_proba(X_train)[:, 1]
y_prob_test = pipeline.predict_proba(X_test)[:, 1]

from sklearn.metrics import roc_auc_score

with mlflow.start_run(run_name="XGBoost_Model") as run:
    mlflow.sklearn.log_model(
        sk_model=pipeline,
        artifact_path="xgboost_pipeline_model",
        registered_model_name="XGBoostPipelineModel"
    )
    
    mlflow.log_param("n_estimators", 500)
    mlflow.log_param("max_depth", 5)
    mlflow.log_param("learning_rate", 0.05)
    mlflow.log_param("subsample", 0.85)
    mlflow.log_param("colsample_bytree", 0.7)
    mlflow.log_param("reg_alpha", 0.1)
    mlflow.log_param("reg_lambda", 1.0)
    mlflow.log_param("random_state", 42)
    mlflow.log_param("tree_method", "hist")
    mlflow.log_param("n_jobs", -1)

    roc_auc_train = roc_auc_score(y_train, y_prob_train)
    roc_auc_test = roc_auc_score(y_test, y_prob_test)

    mlflow.log_metric("roc_auc_train", roc_auc_train)
    mlflow.log_metric("roc_auc_test", roc_auc_test)
    
    print("ROC AUC Score Train:", roc_auc_train)
    print("ROC AUC Score Test:", roc_auc_test)

2025/04/27 15:16:56 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'XGBoostPipelineModel' already exists. Creating a new version of this model...
2025/04/27 15:16:58 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGBoostPipelineModel, version 3
Created version '3' of model 'XGBoostPipelineModel'.


ROC AUC Score Train: 0.9437339618755124
ROC AUC Score Test: 0.8956234734158335
🏃 View run XGBoost_Model at: https://dagshub.com/Givi-Modebadze/Fraud_Detection_ML.mlflow/#/experiments/3/runs/892ef42309cf470ea38618c4ed05596b
🧪 View experiment at: https://dagshub.com/Givi-Modebadze/Fraud_Detection_ML.mlflow/#/experiments/3


# Training 3

In [43]:
from sklearn.pipeline import Pipeline
import xgboost as xgb
import mlflow.sklearn

data_cleaner = mlflow.sklearn.load_model('runs:/6e037bf711a547e49adb76431a26a2ad/data_cleaner_model')
feature_engineer = mlflow.sklearn.load_model('runs:/ea7d88809c464b26aae00605830abd3b/feature_engineer_model')

pipeline = Pipeline([
    ('cleaner', data_cleaner),
    ('feature_engineering', feature_engineer),
    ('xgboost', xgb.XGBClassifier(
        n_estimators=500,
        max_depth=6,
        learning_rate=0.02,
        subsample=0.85,
        colsample_bytree=0.7,
        reg_alpha=0.1,
        reg_lambda=1.0,
        random_state=42,
        n_jobs=-1,
        tree_method="hist",
        use_label_encoder=False,
        eval_metric='auc'
    ))
])

pipeline.fit(X_train, y_train)
y_prob_train = pipeline.predict_proba(X_train)[:, 1]
y_prob_test = pipeline.predict_proba(X_test)[:, 1]

from sklearn.metrics import roc_auc_score

with mlflow.start_run(run_name="XGBoost_Model") as run:
    mlflow.sklearn.log_model(
        sk_model=pipeline,
        artifact_path="xgboost_pipeline_model",
        registered_model_name="XGBoostPipelineModel"
    )
    
    mlflow.log_param("n_estimators", 500)
    mlflow.log_param("max_depth", 6)
    mlflow.log_param("learning_rate", 0.02)
    mlflow.log_param("subsample", 0.85)
    mlflow.log_param("colsample_bytree", 0.7)
    mlflow.log_param("reg_alpha", 0.1)
    mlflow.log_param("reg_lambda", 1.0)
    mlflow.log_param("random_state", 42)
    mlflow.log_param("tree_method", "hist")
    mlflow.log_param("n_jobs", -1)

    roc_auc_train = roc_auc_score(y_train, y_prob_train)
    roc_auc_test = roc_auc_score(y_test, y_prob_test)

    mlflow.log_metric("roc_auc_train", roc_auc_train)
    mlflow.log_metric("roc_auc_test", roc_auc_test)
    
    print("ROC AUC Score Train:", roc_auc_train)
    print("ROC AUC Score Test:", roc_auc_test)

2025/04/27 15:29:32 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'XGBoostPipelineModel' already exists. Creating a new version of this model...
2025/04/27 15:29:35 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGBoostPipelineModel, version 5
Created version '5' of model 'XGBoostPipelineModel'.


ROC AUC Score Train: 0.9358757742727604
ROC AUC Score Test: 0.8923075848730573
🏃 View run XGBoost_Model at: https://dagshub.com/Givi-Modebadze/Fraud_Detection_ML.mlflow/#/experiments/3/runs/14ce8d6600db41d592ab145b66155f3f
🧪 View experiment at: https://dagshub.com/Givi-Modebadze/Fraud_Detection_ML.mlflow/#/experiments/3


# Training 4

In [45]:
from sklearn.pipeline import Pipeline
import xgboost as xgb
import mlflow.sklearn

data_cleaner = mlflow.sklearn.load_model('runs:/6e037bf711a547e49adb76431a26a2ad/data_cleaner_model')
feature_engineer = mlflow.sklearn.load_model('runs:/ea7d88809c464b26aae00605830abd3b/feature_engineer_model')

pipeline = Pipeline([
    ('cleaner', data_cleaner),
    ('feature_engineering', feature_engineer),
    ('xgboost', xgb.XGBClassifier(
        n_estimators=5000,
        max_depth=6,
        learning_rate=0.02,
        subsample=0.85,
        colsample_bytree=0.7,
        reg_alpha=0.5,
        reg_lambda=2.0,
        min_child_weight=10,
        random_state=42,
        n_jobs=-1,
        tree_method="hist",
        use_label_encoder=False,
        eval_metric='auc'
    ))
])


pipeline.fit(X_train, y_train)
y_prob_train = pipeline.predict_proba(X_train)[:, 1]
y_prob_test = pipeline.predict_proba(X_test)[:, 1]

from sklearn.metrics import roc_auc_score

with mlflow.start_run(run_name="XGBoost_Model") as run:
    mlflow.sklearn.log_model(
        sk_model=pipeline,
        artifact_path="xgboost_pipeline_model",
        registered_model_name="XGBoostPipelineModel"
    )
    
    mlflow.log_params({
        "n_estimators": 5000,
        "max_depth": 6,
        "learning_rate": 0.02,
        "subsample": 0.85,
        "colsample_bytree": 0.7,
        "reg_alpha": 0.5,
        "reg_lambda": 2.0,
        "min_child_weight": 10,
        "random_state": 42,
        "tree_method": "hist",
        "n_jobs": -1
    })

    roc_auc_train = roc_auc_score(y_train, y_prob_train)
    roc_auc_test = roc_auc_score(y_test, y_prob_test)

    mlflow.log_metric("roc_auc_train", roc_auc_train)
    mlflow.log_metric("roc_auc_test", roc_auc_test)
    
    print("ROC AUC Score Train:", roc_auc_train)
    print("ROC AUC Score Test:", roc_auc_test)

2025/04/27 15:53:38 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'XGBoostPipelineModel' already exists. Creating a new version of this model...
2025/04/27 15:53:40 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGBoostPipelineModel, version 6
Created version '6' of model 'XGBoostPipelineModel'.


ROC AUC Score Train: 0.9850090402717172
ROC AUC Score Test: 0.885268521145924
🏃 View run XGBoost_Model at: https://dagshub.com/Givi-Modebadze/Fraud_Detection_ML.mlflow/#/experiments/3/runs/e1376b56c53d4c51a04e6d13da3cb79d
🧪 View experiment at: https://dagshub.com/Givi-Modebadze/Fraud_Detection_ML.mlflow/#/experiments/3


# Training 5

In [46]:
from sklearn.pipeline import Pipeline
import xgboost as xgb
import mlflow.sklearn

data_cleaner = mlflow.sklearn.load_model('runs:/6e037bf711a547e49adb76431a26a2ad/data_cleaner_model')
feature_engineer = mlflow.sklearn.load_model('runs:/ea7d88809c464b26aae00605830abd3b/feature_engineer_model')

pipeline = Pipeline([
    ('cleaner', data_cleaner),
    ('feature_engineering', feature_engineer),
    ('xgboost', xgb.XGBClassifier(
        n_estimators=500,
        max_depth=5,
        learning_rate=0.02,
        subsample=0.80,
        colsample_bytree=0.7,
        reg_alpha=0.2,
        reg_lambda=1.0,
        random_state=42,
        n_jobs=-1,
        tree_method="hist",
        use_label_encoder=False,
        eval_metric='auc'
    ))
])

pipeline.fit(X_train, y_train)
y_prob_train = pipeline.predict_proba(X_train)[:, 1]
y_prob_test = pipeline.predict_proba(X_test)[:, 1]

from sklearn.metrics import roc_auc_score

with mlflow.start_run(run_name="XGBoost_Model") as run:
    mlflow.sklearn.log_model(
        sk_model=pipeline,
        artifact_path="xgboost_pipeline_model",
        registered_model_name="XGBoostPipelineModel"
    )
    
    mlflow.log_param("n_estimators", 500)
    mlflow.log_param("max_depth", 5)
    mlflow.log_param("learning_rate", 0.02)
    mlflow.log_param("subsample", 0.80)
    mlflow.log_param("colsample_bytree", 0.7)
    mlflow.log_param("reg_alpha", 0.2)
    mlflow.log_param("reg_lambda", 1.0)
    mlflow.log_param("random_state", 42)
    mlflow.log_param("tree_method", "hist")
    mlflow.log_param("n_jobs", -1)

    roc_auc_train = roc_auc_score(y_train, y_prob_train)
    roc_auc_test = roc_auc_score(y_test, y_prob_test)

    mlflow.log_metric("roc_auc_train", roc_auc_train)
    mlflow.log_metric("roc_auc_test", roc_auc_test)
    
    print("ROC AUC Score Train:", roc_auc_train)
    print("ROC AUC Score Test:", roc_auc_test)

2025/04/27 15:59:47 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'XGBoostPipelineModel' already exists. Creating a new version of this model...
2025/04/27 15:59:50 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGBoostPipelineModel, version 7
Created version '7' of model 'XGBoostPipelineModel'.


ROC AUC Score Train: 0.9205869475656246
ROC AUC Score Test: 0.8892148096536052
🏃 View run XGBoost_Model at: https://dagshub.com/Givi-Modebadze/Fraud_Detection_ML.mlflow/#/experiments/3/runs/e39aeb59c65d499da4df2ca86ec00327
🧪 View experiment at: https://dagshub.com/Givi-Modebadze/Fraud_Detection_ML.mlflow/#/experiments/3


# Training 6

In [49]:
from sklearn.pipeline import Pipeline
import xgboost as xgb
import mlflow.sklearn

data_cleaner = mlflow.sklearn.load_model('runs:/6e037bf711a547e49adb76431a26a2ad/data_cleaner_model')
feature_engineer = mlflow.sklearn.load_model('runs:/ea7d88809c464b26aae00605830abd3b/feature_engineer_model')

pipeline = Pipeline([
    ('cleaner', data_cleaner),
    ('feature_engineering', feature_engineer),
    ('xgboost', xgb.XGBClassifier(
        n_estimators=500,
        max_depth=5,
        learning_rate=0.02,
        subsample=0.85,
        colsample_bytree=0.7,
        reg_alpha=0.1,
        reg_lambda=1.0,
        random_state=42,
        n_jobs=-1,
        tree_method="hist",
        use_label_encoder=False,
        eval_metric='auc'
    ))
])

pipeline.fit(X_train, y_train)
y_prob_train = pipeline.predict_proba(X_train)[:, 1]
y_prob_test = pipeline.predict_proba(X_test)[:, 1]

from sklearn.metrics import roc_auc_score

with mlflow.start_run(run_name="XGBoost_Model") as run:
    mlflow.sklearn.log_model(
        sk_model=pipeline,
        artifact_path="xgboost_pipeline_model",
        registered_model_name="XGBoostPipelineModel"
    )
    
    mlflow.log_param("n_estimators", 500)
    mlflow.log_param("max_depth", 5)
    mlflow.log_param("learning_rate", 0.02)
    mlflow.log_param("subsample", 0.85)
    mlflow.log_param("colsample_bytree", 0.7)
    mlflow.log_param("reg_alpha", 0.1)
    mlflow.log_param("reg_lambda", 1.0)
    mlflow.log_param("random_state", 42)
    mlflow.log_param("tree_method", "hist")
    mlflow.log_param("n_jobs", -1)

    roc_auc_train = roc_auc_score(y_train, y_prob_train)
    roc_auc_test = roc_auc_score(y_test, y_prob_test)

    mlflow.log_metric("roc_auc_train", roc_auc_train)
    mlflow.log_metric("roc_auc_test", roc_auc_test)
    
    print("ROC AUC Score Train:", roc_auc_train)
    print("ROC AUC Score Test:", roc_auc_test)

2025/04/27 16:06:24 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'XGBoostPipelineModel' already exists. Creating a new version of this model...
2025/04/27 16:06:26 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGBoostPipelineModel, version 8
Created version '8' of model 'XGBoostPipelineModel'.


ROC AUC Score Train: 0.9207648999026182
ROC AUC Score Test: 0.8898088663048493
🏃 View run XGBoost_Model at: https://dagshub.com/Givi-Modebadze/Fraud_Detection_ML.mlflow/#/experiments/3/runs/9e6ddd5a8218444a9c4f523efefa2025
🧪 View experiment at: https://dagshub.com/Givi-Modebadze/Fraud_Detection_ML.mlflow/#/experiments/3


# Training 7

In [50]:
from sklearn.pipeline import Pipeline
import xgboost as xgb
import mlflow.sklearn

data_cleaner = mlflow.sklearn.load_model('runs:/6e037bf711a547e49adb76431a26a2ad/data_cleaner_model')
feature_engineer = mlflow.sklearn.load_model('runs:/ea7d88809c464b26aae00605830abd3b/feature_engineer_model')

pipeline = Pipeline([
    ('cleaner', data_cleaner),
    ('feature_engineering', feature_engineer),
    ('xgboost', xgb.XGBClassifier(
        n_estimators=1000,
        max_depth=5,
        learning_rate=0.1,
        subsample=0.85,
        colsample_bytree=0.7,
        reg_alpha=0.1,
        reg_lambda=1.0,
        random_state=42,
        n_jobs=-1,
        tree_method="hist",
        use_label_encoder=False,
        eval_metric='auc'
    ))
])

pipeline.fit(X_train, y_train)
y_prob_train = pipeline.predict_proba(X_train)[:, 1]
y_prob_test = pipeline.predict_proba(X_test)[:, 1]

from sklearn.metrics import roc_auc_score

with mlflow.start_run(run_name="XGBoost_Model") as run:
    mlflow.sklearn.log_model(
        sk_model=pipeline,
        artifact_path="xgboost_pipeline_model",
        registered_model_name="XGBoostPipelineModel"
    )
    
    mlflow.log_param("n_estimators", 1000)
    mlflow.log_param("max_depth", 5)
    mlflow.log_param("learning_rate", 0.1)
    mlflow.log_param("subsample", 0.85)
    mlflow.log_param("colsample_bytree", 0.7)
    mlflow.log_param("reg_alpha", 0.1)
    mlflow.log_param("reg_lambda", 1.0)
    mlflow.log_param("random_state", 42)
    mlflow.log_param("tree_method", "hist")
    mlflow.log_param("n_jobs", -1)

    roc_auc_train = roc_auc_score(y_train, y_prob_train)
    roc_auc_test = roc_auc_score(y_test, y_prob_test)

    mlflow.log_metric("roc_auc_train", roc_auc_train)
    mlflow.log_metric("roc_auc_test", roc_auc_test)
    
    print("ROC AUC Score Train:", roc_auc_train)
    print("ROC AUC Score Test:", roc_auc_test)

2025/04/27 16:13:37 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'XGBoostPipelineModel' already exists. Creating a new version of this model...
2025/04/27 16:13:40 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGBoostPipelineModel, version 9
Created version '9' of model 'XGBoostPipelineModel'.


ROC AUC Score Train: 0.9805996800500353
ROC AUC Score Test: 0.8836679330020585
🏃 View run XGBoost_Model at: https://dagshub.com/Givi-Modebadze/Fraud_Detection_ML.mlflow/#/experiments/3/runs/a52fc99aa22041988551e3a806e74740
🧪 View experiment at: https://dagshub.com/Givi-Modebadze/Fraud_Detection_ML.mlflow/#/experiments/3


# Training 8

In [52]:
from sklearn.pipeline import Pipeline
import xgboost as xgb
import mlflow.sklearn

data_cleaner = mlflow.sklearn.load_model('runs:/6e037bf711a547e49adb76431a26a2ad/data_cleaner_model')
feature_engineer = mlflow.sklearn.load_model('runs:/ea7d88809c464b26aae00605830abd3b/feature_engineer_model')

pipeline = Pipeline([
    ('cleaner', data_cleaner),
    ('feature_engineering', feature_engineer),
    ('xgboost', xgb.XGBClassifier(
        n_estimators=500,
        max_depth=5,
        learning_rate=0.1,
        subsample=0.80,
        colsample_bytree=0.7,
        reg_alpha=0.1,
        reg_lambda=1.0,
        random_state=42,
        n_jobs=-1,
        tree_method="hist",
        use_label_encoder=False,
        eval_metric='auc'
    ))
])

pipeline.fit(X_train, y_train)
y_prob_train = pipeline.predict_proba(X_train)[:, 1]
y_prob_test = pipeline.predict_proba(X_test)[:, 1]

from sklearn.metrics import roc_auc_score

with mlflow.start_run(run_name="XGBoost_Model") as run:
    mlflow.sklearn.log_model(
        sk_model=pipeline,
        artifact_path="xgboost_pipeline_model",
        registered_model_name="XGBoostPipelineModel"
    )
    
    mlflow.log_param("n_estimators", 500)
    mlflow.log_param("max_depth", 5)
    mlflow.log_param("learning_rate", 0.1)
    mlflow.log_param("subsample", 0.80)
    mlflow.log_param("colsample_bytree", 0.7)
    mlflow.log_param("reg_alpha", 0.1)
    mlflow.log_param("reg_lambda", 1.0)
    mlflow.log_param("random_state", 42)
    mlflow.log_param("tree_method", "hist")
    mlflow.log_param("n_jobs", -1)

    roc_auc_train = roc_auc_score(y_train, y_prob_train)
    roc_auc_test = roc_auc_score(y_test, y_prob_test)

    mlflow.log_metric("roc_auc_train", roc_auc_train)
    mlflow.log_metric("roc_auc_test", roc_auc_test)
    
    print("ROC AUC Score Train:", roc_auc_train)
    print("ROC AUC Score Test:", roc_auc_test)

2025/04/27 16:28:02 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'XGBoostPipelineModel' already exists. Creating a new version of this model...
2025/04/27 16:28:05 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGBoostPipelineModel, version 10
Created version '10' of model 'XGBoostPipelineModel'.


ROC AUC Score Train: 0.9633058123162956
ROC AUC Score Test: 0.8908687871894844
🏃 View run XGBoost_Model at: https://dagshub.com/Givi-Modebadze/Fraud_Detection_ML.mlflow/#/experiments/3/runs/51704e8765134ee5bfc3a533bfda57ca
🧪 View experiment at: https://dagshub.com/Givi-Modebadze/Fraud_Detection_ML.mlflow/#/experiments/3
